# Write to firestore

In [ ]:
import random
import os

# Sample a random wine bottle
random_ix = random.sample(range(0,622), k=1)[0]
random_filename = f"wine_bottle_{random_ix:05d}.png"

# Combine with full path
image_dir = "https://storage.googleapis.com/wine-flask/labels_on_bottle/"
image_path = f"{image_dir}{random_filename}"
print(image_path)

In [ ]:
import firebase_admin
#import google.cloud
from firebase_admin import credentials, firestore
from google.cloud import firestore_v1

cred = credentials.Certificate("./.keys/wine-flask-b615f90f43d6.json")
#app = firebase_admin.initialize_app(cred)
store = firestore.client()

file_path = "./data/fake/cleaned_gpt_descriptions_2844.csv"
collection_name = "gpt2-xl-outputs"

In [ ]:
firestore.firestore.FieldPath

In [ ]:
random_key = store.collection(collection_name).document().id

In [ ]:
firestore_v1.field_path.FieldPath.document_id()

In [ ]:
store.collection(collection_name).where(firestore_v1.field_path.FieldPath.document_id(), '==', 'Red Wine').get()

In [ ]:
store.collection(collection_name).where(firestore_v1.field_path.FieldPath.document_id(), '>=', random_key).get()

In [ ]:
firestore_v1.field_path.FieldPath.document_id()

In [ ]:
# Create a reference to the cities collection
cities_ref = db.collection(u'cities')

# Create a query against the collection
query_ref = cities_ref.where(u'state', u'==', u'CA')

In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv(file_path).reset_index().drop(['Unnamed: 0'], axis=1)
print(df.shape)

In [ ]:
def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
for batched_data in batch_data(df, 499):
    batch = store.batch()
    for data_item in batched_data.iterrows():
        doc_ref = store.collection(collection_name).document()
        batch.set(doc_ref, data_item[1].to_dict())
    batch.commit()

In [ ]:
data_item[1].to_dict()

-----------

# Sample from firestore

In [1]:
from firebase_admin import credentials, firestore, initialize_app
from google.cloud import firestore_v1

cred = credentials.Certificate("../.keys/wine-flask-b615f90f43d6.json")
app = initialize_app(cred)
store = firestore.client()

#file_path = "./data/fake/cleaned_gpt_descriptions_2844.csv"
collection_name = "gpt2-xl-outputs"

In [2]:
random_key = store.collection(collection_name).document().id
random_key

'eBbs6mm7x8CoBF0j9cn4'

In [3]:
%%time


result = store.collection(collection_name).where('id', '>=', random_key).limit(1).get()[0]
result.to_dict()

CPU times: user 41.4 ms, sys: 1.35 ms, total: 42.7 ms
Wall time: 536 ms


{'id': 'eCGFGHRs0ft0Gh2GGDe7',
 'name': 'Kins Bernacher Resard Chassornay Merlot 1996',
 'category_1': 'Red Wine',
 'origin': 'Burgundy, France',
 'category_2': 'Merlot',
 'description': 'The deep, ruby color of the wine is a prelude to the dense, rich flavors of black cherries, grade-grown strawberries and coffee.  The mouthfeel is soft and sweet, with good balance between sweetness and firm tannins.  The finish is long and pleasing.'}